In [ ]:
# Step 1: Install and Load Required Libraries
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 102.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import pandas as pd
from spacy.training import Example

# Step 2: Load the Pretrained spaCy Model
nlp = spacy.load("en_core_web_sm", exclude=["lookups"])

# Step 3: Load and Preprocess the Data
# Load training dataset
train_df = pd.read_csv('/content/sample_data/Train_1.csv')

# Drop rows with missing text or location
train_df = train_df.dropna(subset=['text', 'location'])

# Step 4: Extract Locations using spaCy's NER
def extract_locations(text):
    doc = nlp(text)
    # Extract locations labeled as GPE, LOC, FAC (geopolitical entities, locations, facilities)
    locations = [ent.text for ent in doc.ents if ent.label_ in ['GPE', 'LOC', 'FAC']]
    return " ".join(locations)

# Apply spaCy NER on training data to extract locations
train_df['predicted_locations'] = train_df['text'].apply(extract_locations)

# Display a few samples
print(train_df[['tweet_id', 'text', 'location', 'predicted_locations']].head())

# Step 5: Fine-Tuning spaCy's NER Model (Optional)
# Prepare training data for spaCy
train_data = []

def get_non_overlapping_entities(doc, locations):
    """Helper function to prevent overlapping entities."""
    used_spans = []
    entities = []
    for loc in locations.split():
        start = doc.find(loc)
        if start != -1:
            end = start + len(loc)
            overlap = False
            # Check for overlap with previously added entities
            for span in used_spans:
                if not (end <= span[0] or start >= span[1]):
                    overlap = True
                    break
            if not overlap:
                entities.append((start, end, 'GPE'))  # Mark location as GPE
                used_spans.append((start, end))
    return entities

for _, row in train_df.iterrows():
    doc = row['text']
    annotations = {'entities': get_non_overlapping_entities(doc, row['location'])}
    train_data.append((doc, annotations))

# Fine-tune the NER model
ner = nlp.get_pipe('ner')
for _, annotations in train_data:
    for ent in annotations['entities']:
        ner.add_label(ent[2])

# Disable other pipes and only train the NER component
optimizer = nlp.resume_training()  # Use resume_training instead of initialize

for itn in range(10):  # Number of iterations
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], sgd=optimizer)

# Save the fine-tuned model
nlp.to_disk('fine_tuned_ner_model')




/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


                 tweet_id                                               text  \
1  ID_1001136696589631488  Flash floods struck a Maryland city on Sunday,...   
2  ID_1001136950345109504  State of emergency declared for Maryland flood...   
3  ID_1001137334056833024  Other parts of Maryland also saw significant d...   
4  ID_1001138374923579392  Catastrophic Flooding Slams Ellicott City, Mar...   
5  ID_1001138377717157888  WATCH: 1 missing after flash #FLOODING devasta...   

                 location                  predicted_locations  
1                Maryland                             Maryland  
2                Maryland                             Maryland  
3      Baltimore Maryland  Maryland Baltimore Dundalk Maryland  
4  Ellicott City Maryland                             Maryland  
5  Ellicott City Maryland               Ellicott City Maryland  


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "#Flashflood tears out street poles on #EllicottCit..." with entities "[(193, 202, 'GPE'), (39, 47, 'GPE'), (47, 51, 'GPE...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "3 Days Greek National Mourning where at least 60 p..." with entities "[(102, 108, 'GPE'), (130, 136, 'GPE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the 

In [ ]:
# Step 6: Load Test Data and Predict Locations
test_df = pd.read_csv('/content/sample_data/Test.csv')

# Apply the fine-tuned NER model on test data
test_df['predicted_locations'] = test_df['text'].apply(extract_locations)

# Step 7: Create Submission File
submission = test_df[['tweet_id', 'predicted_locations']].rename(columns={'predicted_locations': 'location'})
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

Submission file created successfully!


In [ ]:
# prompt: in submission.csv file  column location value null replace it with most occuring location

import pandas as pd

# Load the submission file
submission = pd.read_csv('submission.csv')

# Find the most occurring location
most_frequent_location = submission['location'].mode()[0]

# Replace null values with the most frequent location
submission['location'].fillna(most_frequent_location, inplace=True)

# Save the updated submission file
submission.to_csv('submission.csv', index=False)

print("Submission file updated successfully!")


Submission file updated successfully!


<ipython-input-10-a834068cd41c>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  submission['location'].fillna(most_frequent_location, inplace=True)
